In [1]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem

from keras import backend as K
from keras.models import load_model
import tensorflow as tf

from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)

image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

using canonical datasplit..


In [15]:
ytr_deg

array([ 210.,  316.,  319., ...,  193.,  186.,  179.])

In [14]:
import pickle


array([ 151.,  145.,  150.,  153.,  142.,  177.,  293.,  283.,  132.,
        144.,  176.,  138.,  143.,  341.,  347.,    7.,  173.,   36.,
         32.,  340.,  176.,  349.,  345.,  186.,  173.,  177.,  180.,
        180.,  180.,  175.,  167.,    0.,  329.,  312.,  140.,  147.,
        147.,  324.,  330.,  347.,  233.,  133.,  127.,  326.,  316.,
        173.,  161.,  170.,  149.,  158.,  329.,  321.,  347.,  302.,
        117.,  287.,  295.,  299.,  136.,  129.,  171.,  141.,  199.,
        353.,  347.,  346.,  353.,  160.,  349.,  355.,  145.,  152.,
        163.,  173.,  175.,  297.,  300.,  302.,  299.,  104.,  119.,
        123.,  152.,  156.,  314.,  164.,  146.,  125.,  207.,  181.,
        152.,  143.,  152.,  343.,  338.,  153.,  154.,  148.,  176.,
          3.,  332.,  334.,  356.,  341.,  314.,  320.,  320.,  318.,
        343.,  338.,    7.,  349.,  353.,  347.,   53.,  294.,  327.,
        350.,  145.,  139.,  136.,  142.,   37.,  356.,    9.,  154.,
        148.,  328.,

#### Loading pre-trained models

In [8]:
vgg_fixed_kappa = vgg.BiternionVGG(image_height=image_height,
                                   image_width=image_width,
                                   n_channels=3,
                                   predict_kappa=False,
                                   fixed_kappa_value=3.0)

vgg_fixed_kappa.model.load_weights('/Users/sergey/biternion_net_cluster_logs/vggbit_likelihood_learned_kappa_Adadelta_200epochs_batch10_89dd8240ed8b2175ad49/vgg_bit_vm_likelihood_town.best.weights.h5')

In [9]:
results = dict()
results['train'] = vgg_fixed_kappa.evaluate(xtr, ytr_deg, 'train')
results['validation'] = vgg_fixed_kappa.evaluate(xval, yval_deg, 'validation')
results['test'] = vgg_fixed_kappa.evaluate(xte, yte_deg, 'test')

MAAD error (train) : 71.467049 ± 57.111773
log-likelihood (train) : -2.663717 ± 0.027228SEM
MAAD error (validation) : 65.314157 ± 56.263639
log-likelihood (validation) : -2.413951 ± 0.076771SEM
MAAD error (test) : 67.156941 ± 56.452665
log-likelihood (test) : -2.491740 ± 0.071362SEM


In [3]:
vgg_learned_kappa = vgg.BiternionVGG(image_height=image_height,
                                     image_width=image_width,
                                     n_channels=3,
                                     predict_kappa=True)

vgg_learned_kappa.model.load_weights('/Users/sergey/BiternionNet/logs/vggbit_likelihood_learned_kappa_Adadelta_200epochs_batch10_374f90e721d0fbc523ea/vgg_bit_vm_likelihood_town.best.weights.h5')

ValueError: You are trying to load a weight file containing 2 layers into a model with 3 layers.

In [60]:
learned_tr_res = eval_bvgg_model(vgg_learned_kappa, xtr, ytr_deg, ytr_bit, 'train')
learned_val_res = eval_bvgg_model(vgg_learned_kappa, xval, yval_deg, yval_bit, 'validation')
learned_te_res = eval_bvgg_model(vgg_learned_kappa, xte, yte_deg, yte_bit, 'test')
learned_te_preds_bit = vgg_fixed_kappa.model.predict(xte)
learned_te_preds_deg = bit2deg(learned_te_preds_bit)

MAAD error (train) : 24.852653 ± 30.807507
log-likelihood (train) : -0.745141 ± 0.012439SEM
MAAD error (validation) : 25.865987 ± 34.247103
log-likelihood (validation) : -0.731408 ± 0.036385SEM
MAAD error (test) : 26.618819 ± 33.802968
log-likelihood (test) : -0.744777 ± 0.033247SEM


In [21]:
vgg_cosine = vgg.BiternionVGG(image_height=image_height,
                              image_width=image_width,
                              n_channels=3,
                              predict_kappa=False)

vgg_cosine.model.load_weights('logs/best_models/cosine/vgg_bit_cosine_town.final_model.h5')

In [29]:
cosine_tr_res = eval_bvgg_model(vgg_cosine, xtr, ytr_deg, ytr_bit, 'train')
cosine_val_res = eval_bvgg_model(vgg_cosine, xval, yval_deg, yval_bit, 'validation')
cosine_te_res = eval_bvgg_model(vgg_cosine, xte, yte_deg, yte_bit, 'test')
learned_te_preds_bit = vgg_cosine.model.predict(xte)
learned_te_preds_deg = bit2deg(fixed_te_preds_bit)

MAAD error (train) : 13.700977 ± 17.924503
predicted kappa (train) : 1.000000 ± 0.000000
log-likelihood (train) : -1.135066 ± 0.002367SEM
MAAD error (validation) : 14.782664 ± 19.419558
predicted kappa (validation) : 1.000000 ± 0.000000
log-likelihood (validation) : -1.145701 ± 0.007528SEM
MAAD error (test) : 15.520041 ± 22.154984
predicted kappa (test) : 1.000000 ± 0.000000
log-likelihood (test) : -1.154747 ± 0.008343SEM


In [31]:
vgg_vm = vgg.BiternionVGG(image_height=image_height,
                          image_width=image_width,
                          n_channels=3,
                          predict_kappa=False)

vgg_vm.model.load_weights('logs/best_models/von_mises/vgg_bit_von_mises_town.best.weights.h5')

In [32]:
vm_tr_res = eval_bvgg_model(vgg_vm, xtr, ytr_deg, ytr_bit, 'train')
vm_val_res = eval_bvgg_model(vgg_vm, xval, yval_deg, yval_bit, 'validation')
vm_te_res = eval_bvgg_model(vgg_vm, xte, yte_deg, yte_bit, 'test')

MAAD error (train) : 16.481105 ± 22.488998
predicted kappa (train) : 1.000000 ± 0.000000
log-likelihood (train) : -1.162347 ± 0.003133SEM
MAAD error (validation) : 17.421278 ± 25.500195
predicted kappa (validation) : 1.000000 ± 0.000000
log-likelihood (validation) : -1.176983 ± 0.010404SEM
MAAD error (test) : 16.819883 ± 23.218121
predicted kappa (test) : 1.000000 ± 0.000000
log-likelihood (test) : -1.163565 ± 0.008568SEM


#### CVAE model 

In [12]:
n_u = 8
cvae_best = CVAE(n_hidden_units=n_u)
cvae_ckpt_path = 'logs/cvae.full_model.weights.hdf5'
cvae_best.full_model.load_weights(cvae_ckpt_path)

In [52]:
def eval_cvae_model(cvae_model, x, ytrue_deg, ytrue_bit, data_part):
    
    n_samples = x.shape[0]

    cvae_preds = cvae_model.full_model.predict([x, ytrue_bit])
    elbo_te, ll_te, kl_te = cvae_model._cvae_elbo_loss_np(ytrue_bit, cvae_preds)

    ypreds = cvae_model.decoder_model.predict(x)
    ypreds_bit = ypreds[:,0:2]
    kappa_preds_te = ypreds[:,2:]

    ypreds_deg = bit2deg(ypreds_bit)

    loss_te = maad_from_deg(ytrue_deg, ypreds_deg)
    mean_loss_te = np.mean(loss_te)
    std_loss_te = np.std(loss_te)

    print("MAAD error (%s) : %f ± %f" % (data_part, mean_loss_te, std_loss_te))

    # print("kappa (%s) : %f ± %f" % (data_part, np.mean(kappa_preds_te), np.std(kappa_preds_te)))

    log_likelihood_loss = von_mises_log_likelihood_np(ytrue_bit, ypreds_bit, kappa_preds_te,
                                                         input_type='biternion')

    # print("ELBO (%s) : %f ± %f SEM" % (data_part, np.mean(-elbo_te), sem(-elbo_te)))

    #print("KL(encoder|prior) (%s) : %f ± %f SEM" % (data_part, np.mean(-kl_te), sem(-kl_te)))

    print("log-likelihood (%s) : %f±%fSEM" % (data_part, 
                                              np.mean(log_likelihood_loss), 
                                              sem(log_likelihood_loss)))
    return

In [53]:
eval_cvae_model(cvae_best, xte, yte_deg, yte_bit, 'train')
eval_cvae_model(cvae_best, xval, yval_deg, yval_bit, 'validation')
eval_cvae_model(cvae_best, xte, yte_deg, yte_bit, 'test')

MAAD error (train) : 24.147247 ± 31.294075
log-likelihood (train) : -0.689477±0.054274SEM
MAAD error (validation) : 23.855638 ± 32.482711
log-likelihood (validation) : -0.652637±0.058809SEM
MAAD error (test) : 24.191327 ± 31.100941
log-likelihood (test) : -0.691666±0.054426SEM
